In [2]:
from flask import Flask, request, render_template
import tensorflow as tf
from PIL import Image
from io import BytesIO
import base64
from flask_ngrok import run_with_ngrok
import requests

## Load Model

In [3]:
MODEL_PATH = './models/64/'

model = tf.keras.models.load_model(MODEL_PATH)

## Run App

In [ ]:
app = Flask(__name__)

def predict(img) :
    img_array = tf.keras.utils.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    img_array = tf.image.resize(img_array, [180, 180])
    
    predictions = model.predict(img_array)
    cat_percentage = predictions[0][0] * 100
    dog_percentage = predictions[0][1] * 100
    
    return cat_percentage, dog_percentage

def get_prediction_text(cat_percentage, dog_percentage) :
    max_value = max(cat_percentage, dog_percentage)
    
    animal = "cat" if cat_percentage > dog_percentage else "dog"
    
    if max_value >= 85 :
        return 'definitely a {}'.format(animal)
    elif max_value >= 60 :
        return 'probably a {}'.format(animal)
    else :
        return 'looks like a {}'.format(animal)

def try_get_image_from_file(request) :
    if 'file' not in request.files :
        return False, '', '', 0, 0
    
    file = request.files['file']
    
    if file.filename == '' :
        return False, '', '', 0, 0
    
    if file :
            img = Image.open(file.stream)
            with BytesIO() as buf :
                img.save(buf, 'png')
                image_bytes = buf.getvalue()
                
            encoded_string = base64.b64encode(image_bytes).decode()
        
            cat_percentage, dog_percentage = predict(img)

            return True, encoded_string, get_prediction_text(cat_percentage, dog_percentage), cat_percentage, dog_percentage
    
    return False, '', '', 0, 0

def try_get_image_from_url(request) :
    url = request.form['url']
    
    if url == '' :
        return False, '', '', 0, 0
    
    img = Image.open(requests.get(url, stream=True).raw)
    
    with BytesIO() as buf :
        img.save(buf, 'png')
        image_bytes = buf.getvalue()
        
    encoded_string = base64.b64encode(image_bytes).decode()

    cat_percentage, dog_percentage = predict(img)

    return True, encoded_string, get_prediction_text(cat_percentage, dog_percentage), cat_percentage, dog_percentage

@app.route('/', methods=['GET', 'POST'])
def index():
    if request.method == 'POST' :
        sucess, img_data, prediction_text, cat_percentage, dog_percentage = try_get_image_from_file(request)
        
        if sucess :
            return render_template('index.html', img_data=img_data, prediction_text=prediction_text, cat_percentage=cat_percentage, dog_percentage=dog_percentage), 200
        
        sucess, img_data, prediction_text, cat_percentage, dog_percentage = try_get_image_from_url(request)
            
        return render_template('index.html', img_data=img_data, prediction_text=prediction_text, cat_percentage=cat_percentage, dog_percentage=dog_percentage), 200            
        
    return render_template('index.html', img_data='', prediction_text='', cat_percentage=0, dog_percentage=0), 200

run_with_ngrok(app)

if __name__ == '__main__':
    app.run()